In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.io
import math
import sklearn
import sklearn.datasets
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [0]:
# the dataset given for any phone:-accel or gyro / watch:-accel or gyro couldnt be fitted in a single csv file (for all 20 data in excel stacked above each other) , 
# so I loaded first 14 data set in 1 file and the rest in the 2nd file
def load_data():
    dataset = pd.read_csv('Train1 FILE HERE.csv', header=None)
    train_set_x_orig1 = dataset.iloc[:,[3,4,5]].values # your train set features
    train_set_y_orig1 = dataset.iloc[:,[1]].values

    dataset = pd.read_csv('Train2 FILE HERE.csv', header=None)
    train_set_x_orig2 = dataset.iloc[:,[3,4,5]].values # your train set features
    train_set_y_orig2 = dataset.iloc[:,[1]].values

    train_set_x_orig = np.vstack((train_set_x_orig1,train_set_x_orig2)) # stacking the two csv file one over the other in numpy array
    train_set_y_orig = np.vstack((train_set_y_orig1,train_set_y_orig2))

    train_x = train_set_x_orig.T
    train_y = train_set_y_orig
    onehotencoder=OneHotEncoder()
    train_y= onehotencoder.fit_transform(train_y[:,[0]]).toarray()
    train_y = train_y.T

    dataset = pd.read_csv('Test1 FILE HERE.csv', header=None)
    test_set_x_orig1 = dataset.iloc[:,[3,4,5]].values # taking the useful train set features
    test_set_y_orig1 = dataset.iloc[:,[1]].values

    dataset = pd.read_csv('Test1 FILE HERE.csv', header=None)
    test_set_x_orig2 = dataset.iloc[:,[3,4,5]].values # taking the useful train set features
    test_set_y_orig2 = dataset.iloc[:,[1]].values

    test_set_x_orig = np.vstack((test_set_x_orig1,test_set_x_orig2)) # stacking the two csv file one over the other in numpy array
    test_set_y_orig = np.vstack((test_set_y_orig1,test_set_y_orig2))

    test_x = test_set_x_orig.T
    test_y = test_set_y_orig
    onehotencoder=OneHotEncoder()
    test_y= onehotencoder.fit_transform(test_y[:,[0]]).toarray()
    test_y = test_y.T

    return train_x,train_y,test_x,test_y

In [0]:
dict = {0 :'A' ,
        1 :'B' ,
        2 :'C' ,
        3 :'D' ,
        4 :'E' ,
        5 :'F' ,
        6 :'G' ,
        7 :'H' ,
        8 :'I' ,
        9 :'J' ,
        10 :'K' ,
        11:'L' ,
        12 :'M' ,
        13:'O' ,
        14:'P' ,
        15:'Q' ,
        16:'R' ,
        17:'S' }

In [0]:
def softmax(X):
    
    #A -- output of sigmoid(z), same shape as Z

    
    x_exp = np.exp(X)
    x_sum = np.sum(x_exp,axis=0,keepdims=True)
    A = x_exp/x_sum
    
    return A

In [0]:
def relu(X):
  

    #A -- Post-activation parameter, of the same shape as Z

    
    A = np.maximum(0,X)
    
    return A

In [0]:
def initialize_parameters(layer_dims):

    np.random.seed(3)
    parameters = {}
    L = len(layer_dims) # number of layers in the network

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1])*  np.sqrt(2 / layer_dims[l-1])
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
        assert parameters['W' + str(l)].shape[0] == layer_dims[l], layer_dims[l-1]
        assert parameters['W' + str(l)].shape[0] == layer_dims[l], 1
        
    return parameters

In [0]:
def compute_cost(a4, Y):

    logprobs = np.multiply(-np.log(a4),Y) + np.multiply(-np.log(1 - a4), 1 - Y)
    cost_total =  np.sum(logprobs)
    
    return cost_total

In [0]:
def forward_propagation(X, parameters):
 
    # retrieve parameters
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]
    W4 = parameters["W4"]
    b4 = parameters["b4"]
    
    # LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID
    z1 = np.dot(W1, X) + b1
    a1 = relu(z1)
    z2 = np.dot(W2, a1) + b2
    a2 = relu(z2)
    z3 = np.dot(W3, a2) + b3
    a3 = relu(z3)
    z4 = np.dot(W4, a3) + b4
    a4 = softmax(z4)
    
    cache = (z1, a1, W1, b1, z2, a2, W2, b2, z3, a3, W3, b3, z4, a4, W4, b4)
    
    return a4, cache

In [0]:
def backward_propagation(X, Y, cache):

    m = X.shape[1]
    (z1, a1, W1, b1, z2, a2, W2, b2, z3, a3, W3, b3, z4, a4, W4, b4) = cache
    
    dz4 = 1./m * (a4 - Y)
    dW4 = np.dot(dz4, a3.T)
    db4 = np.sum(dz4, axis=1, keepdims = True)
    
    da3 = np.dot(W4.T, dz4)
    dz3 = np.multiply(da3, np.int64(a3 > 0))
    dW3 = np.dot(dz3, a1.T)
    db3 = np.sum(dz3, axis=1, keepdims = True)
    
    da2 = np.dot(W3.T, dz3)
    dz2 = np.multiply(da2, np.int64(a2 > 0))
    dW2 = np.dot(dz2, a1.T)
    db2 = np.sum(dz2, axis=1, keepdims = True)
    
    da1 = np.dot(W2.T, dz2)
    dz1 = np.multiply(da1, np.int64(a1 > 0))
    dW1 = np.dot(dz1, X.T)
    db1 = np.sum(dz1, axis=1, keepdims = True)
    
    gradients = {"dz4": dz4, "dW4": dW4, "db4": db4,
                 "dz3": dz3, "dW3": dW3, "db3": db3, "db3": db3,
                 "da2": da2, "dz2": dz2, "dW2": dW2, "db2": db2,
                 "da1": da1, "dz1": dz1, "dW1": dW1, "db1": db1}
    
    return gradients

In [0]:
def predict(X, y, parameters):

    
    m = X.shape[1]
    p = np.zeros((1,m), dtype = np.int)
    
    # Forward propagation
    a4, caches = forward_propagation(X, parameters)
    

    for i in range(0, a4.shape[1]):
      new_list = list(a4[:,i])
      maxm = new_list.index(max(new_list))
      a4[maxm,i]=1
    a4 = (a4>=1).astype(int)
    
    prob = a4.T
    y = y.T
    incorrect = np.sum(y!=prob)/2
    accu = (m-incorrect)/m

    print("Accuracy: "  +  str(accu))
    
    return None

In [0]:
def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):

    
    np.random.seed(seed)            # To make your "random" minibatches the same as ours
    m = X.shape[1]                  # number of training examples
    mini_batches = []
        

    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((18,m))


    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):

        mini_batch_X = shuffled_X[:,k * mini_batch_size:(k + 1) * mini_batch_size]
        mini_batch_Y = shuffled_Y[:,k * mini_batch_size:(k + 1) * mini_batch_size]

        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:

        mini_batch_X = shuffled_X[:,num_complete_minibatches * mini_batch_size:]
        mini_batch_Y = shuffled_Y[:,num_complete_minibatches * mini_batch_size:]

        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

In [0]:
def initialize_adam(parameters) :

    
    L = len(parameters) // 2 # number of layers in the neural networks
    v = {}
    s = {}
    
    # Initialize v, s. Input: "parameters". Outputs: "v, s".
    for l in range(L):

        v["dW" + str(l + 1)] = np.zeros_like(parameters["W" + str(l + 1)])
        v["db" + str(l + 1)] = np.zeros_like(parameters["b" + str(l + 1)])
        s["dW" + str(l+1)] = np.zeros_like(parameters["W" + str(l + 1)])
        s["db" + str(l+1)] = np.zeros_like(parameters["b" + str(l + 1)])

    
    return v, s

In [0]:
def update_parameters_with_adam(parameters, grads, v, s, t, learning_rate=0.01,
                                beta1=0.9, beta2=0.999, epsilon=1e-8):

    L = len(parameters) // 2                 # number of layers in the neural networks
    v_corrected = {}                         # Initializing first moment estimate, python dictionary
    s_corrected = {}                         # Initializing second moment estimate, python dictionary
    
    # Perform Adam update on all parameters
    for l in range(L):
        # Moving average of the gradients. Inputs: "v, grads, beta1". Output: "v".
 
        v["dW" + str(l + 1)] = beta1 * v["dW" + str(l + 1)] + (1 - beta1) * grads['dW' + str(l + 1)]
        v["db" + str(l + 1)] = beta1 * v["db" + str(l + 1)] + (1 - beta1) * grads['db' + str(l + 1)]


        # Compute bias-corrected first moment estimate. Inputs: "v, beta1, t". Output: "v_corrected".

        v_corrected["dW" + str(l + 1)] = v["dW" + str(l + 1)] / (1 - np.power(beta1, t))
        v_corrected["db" + str(l + 1)] = v["db" + str(l + 1)] / (1 - np.power(beta1, t))


        # Moving average of the squared gradients. Inputs: "s, grads, beta2". Output: "s".

        s["dW" + str(l + 1)] = beta2 * s["dW" + str(l + 1)] + (1 - beta2) * np.power(grads['dW' + str(l + 1)], 2)
        s["db" + str(l + 1)] = beta2 * s["db" + str(l + 1)] + (1 - beta2) * np.power(grads['db' + str(l + 1)], 2)


        # Compute bias-corrected second raw moment estimate. Inputs: "s, beta2, t". Output: "s_corrected".

        s_corrected["dW" + str(l + 1)] = s["dW" + str(l + 1)] / (1 - np.power(beta2, t))
        s_corrected["db" + str(l + 1)] = s["db" + str(l + 1)] / (1 - np.power(beta2, t))


        # Update parameters. Inputs: "parameters, learning_rate, v_corrected, s_corrected, epsilon". Output: "parameters".

        parameters["W" + str(l + 1)] = parameters["W" + str(l + 1)] - learning_rate * v_corrected["dW" + str(l + 1)] / np.sqrt(s_corrected["dW" + str(l + 1)] + epsilon)
        parameters["b" + str(l + 1)] = parameters["b" + str(l + 1)] - learning_rate * v_corrected["db" + str(l + 1)] / np.sqrt(s_corrected["db" + str(l + 1)] + epsilon)


    return parameters, v, s

In [0]:
train_x, train_y, test_x, test_y= load_data()

In [48]:
#standardise here

sc = StandardScaler()

train_x =train_x.T
test_x = test_x.T

train_x[:, :] = sc.fit_transform(train_x[:, :])
test_x[:, :] = sc.transform(test_x[:, :])

train_x =train_x.T
test_x = test_x.T

print ("train_x's shape: " + str(train_x.shape))
print ("test_x's shape: " + str(test_x.shape))


train_x's shape: (3, 1580245)
test_x's shape: (3, 111026)


In [0]:
def model(X, Y, layers_dims, learning_rate = 0.0007, mini_batch_size = 64, beta = 0.9,
          beta1 = 0.9, beta2 = 0.999,  epsilon = 1e-8, num_epochs = 10000, print_cost = True):

    L = len(layers_dims)             # number of layers in the neural networks
    costs = []                       # to keep track of the cost
    t = 0                            # initializing the counter required for Adam update
    seed = 10                        # For grading purposes, so that your "random" minibatches are the same as ours
    m = X.shape[1]                   # number of training examples
    
    # Initialize parameters
    parameters = initialize_parameters(layers_dims)

    v, s = initialize_adam(parameters)
    
    # Optimization loop
    for i in range(num_epochs):
        
        # Define the random minibatches. We increment the seed to reshuffle differently the dataset after each epoch
        seed = seed + 1
        minibatches = random_mini_batches(X, Y, mini_batch_size, seed)
        cost_total = 0
        
        for minibatch in minibatches:

            # Select a minibatch
            (minibatch_X, minibatch_Y) = minibatch

            # Forward propagation
            a4, caches = forward_propagation(minibatch_X, parameters)

            # Compute cost and add to the cost total
            cost_total += compute_cost(a4, minibatch_Y)

            # Backward propagation
            grads = backward_propagation(minibatch_X, minibatch_Y, caches)

            t = t + 1 # Adam counter
            parameters, v, s = update_parameters_with_adam(parameters, grads, v, s, t, learning_rate, beta1, beta2,  epsilon)
        
        cost_avg = cost_total / m
        
        # Print the cost every 1000 epoch
        if print_cost and i % 1000 == 0:
            print ("Cost after epoch %i: %f" %(i, cost_avg))
        if print_cost and i % 100 == 0:
            costs.append(cost_avg)

    return parameters

In [50]:
#model train
layers_dims = [3, 30, 25, 25, 18] #  4-layer model
parameters = model(train_x, train_y, layers_dims, num_epochs = 200, print_cost = True)

ValueError: ignored

In [0]:
pred_train = predict(train_x, train_y, parameters)

In [0]:
pred_test = predict(test_x, test_y, parameters)